In [1]:
import requests
from bs4 import BeautifulSoup as bs
from selenium import webdriver
import time
import pandas as pd
import json
from pathlib import Path

In [ ]:
#need to install the chromedriver.exe in the Anaconda3/scripts folder or you will get a path error
driver = webdriver.Chrome()

#fetch url with selenium using chrome looping through to page 18, after that there are no school ratings
school_name_list = []
school_street_list = []
school_zip_list = []
school_city_list = []
school_rating_list = []
page_num = 1

while page_num < 19:
#setting url to get school ratings table that we will scrape.  Will iterate using page_num
    url = f"https://www.greatschools.org/search/search.page?gradeLevels%5B%5D=e&q=houston%20tx&sort=rating&view=table&page={page_num}"
    driver.get(url)
    
#using sleep to ensure page is fully loaded each loop before attempting to scrape    
    time.sleep(8)

#returning selenium html and then switching to beautiful soup
    innerHTML = driver.execute_script("return document.body.innerHTML")
    
    soup = bs(innerHTML,'lxml')
    
#limiting to 25 because the page is duplicating the html then hiding one set.  Was getting 50 out of 25 results

    school_name = soup.find_all('a', class_ ='name',limit=25)
    school_street_zip = soup.find_all('div', class_ ='address',limit=25)
    school_rating = soup.find_all('div', class_ ='circle-rating--small',limit=25)
    
#looping through the above to get text from each element and append to lists

    for name in school_name:
        name = name.text
        school_name_list.append(name)
        
    for street in school_street_zip:
        street = street.text
        school_street_list.append(street[:-20])
        
    for zipcode in school_street_zip:
        zipcode = zipcode.text
        school_zip_list.append(zipcode[-5:])
    
    for city in school_street_zip:
        city = city.text
        school_city_list.append(city[-18:-11])
        
    for rating in school_rating:
        rating = rating.text
        school_rating_list.append(rating[:-3])
    
    page_num += 1

In [5]:
# print len to check if scrape worked properly
print(len(school_city_list))

450


In [21]:
schools_dict = {'School':pd.Series(school_name_list),'Street':pd.Series(school_street_list),'Zip':pd.Series(school_zip_list),'City':pd.Series(school_city_list),'Rating':pd.Series(school_rating_list).astype(int)}
schools_df = pd.DataFrame(schools_dict)
#dropping last 6 rows because they did not have ratings
schools_df = schools_df[:-6]
#remove where city != Houston (ie school has Houston in name but not address)
schools_df = schools_df[schools_df.City =='Houston']
schools_df.index = range(len(schools_df))
schools_df.tail()

,School,Street,Zip,City,Rating
418,Cook Jr Elementary School,7115 Lockwood Drive,77016,Houston,2
419,Admin Services,4250 Cook Road,77072,Houston,1
420,Sneed Elementary School,9855 Pagewood Lane,77042,Houston,1
421,Highland Hts Elementary School,865 Paul Quinn Street,77091,Houston,1
422,Pro-Vision Middle,4590 Wilmington Street,77051,Houston,1


In [7]:
# ###BLOCK COMMENTED OUT BECAUSE IT TAKES APPROX 30 MINUTES TO RUN###
# #Change format of addresses so we can plug into url
# school_street_list_new = schools_df.Street.to_list()

# formatted_address = []
# for i in school_street_list_new:
#     formatted = i.replace(' ','+')
#     formatted_address.append(formatted)


# fips_list = []

# for i in formatted_address:
#     street_url = formatted_address[count]
#     fips_url=f"https://geocoding.geo.census.gov/geocoder/geographies/address?street={street_url}&city=Houston&state=Tx&benchmark=Public_AR_Census2010&vintage=Census2010_Census2010&layers=14&format=json"
#     fips_req = requests.get(fips_url)
#     time.sleep(1)
#     fips_json = json.loads(fips_req.text)
    
#     try:
#         fips_list.append([fips_json][0]['result']['addressMatches'][0]['geographies']['Census Blocks'][0]['GEOID'][:-4])
#     except Exception:
#         fips_list.append('null')
#     
    

# print(len(fips_list))

423


In [9]:
fips_series = pd.Series(fips_list)
fips_series

0      48201430600
1      48201430400
2      48201450100
3      48201450200
4      48201451601
5      48201411400
6      48201412400
7             null
8             null
9      48201520700
10     48201421300
11     48201432001
12     48201332900
13            null
14     48201432600
15     48201340203
16     48201430600
17     48201430300
18     48201510200
19     48201511302
20     48201420700
21     48201531100
22     48201320200
23     48201422000
24     48201412200
25     48201413000
26     48201410200
27     48201431501
28            null
29     48201332900
          ...     
393    48201433600
394    48201522402
395    48201521100
396    48201522302
397    48201520400
398    48201550601
399    48157670101
400    48201432600
401    48201421202
402    48201451300
403    48201422800
404    48201211400
405           null
406    48201331601
407    48201211100
408           null
409    48201332200
410    48201211500
411    48201531600
412    48201530800
413    48201210500
414    48201

In [26]:
schools_df['FIPS'] = fips_series
schools_df.head(50)

,School,Street,Zip,City,Rating,FIPS
0,Frostwood Elementary School,12214 Memorial Drive,77024,Houston,10,48201430600
1,Memorial Drive Elementary School,11202 Smithdale Road,77024,Houston,10,48201430400
2,Rummel Creek Elementary School,625 Brittmoore Road,77079,Houston,10,48201450100
3,Wilchester Elementary School,13618 Saint Marys Lane,77079,Houston,10,48201450200
4,Bush Elementary School,13800 Westerloch Drive,77077,Houston,10,48201451601
5,River Oaks Elementary School,2008 Kirby Drive,77019,Houston,10,48201411400
6,West University Elementary School,3756 University Boulevard,77005,Houston,10,48201412400
7,Shadow Forest Elementary School,2300 Mills Branch Drive,77345,Houston,10,null
8,Willow Creek Elementary School,2002 Willow Terrace Drive,77345,Houston,10,null
9,The School For Highly Gifted Students,8390 Westview Drive,77055,Houston,10,48201520700


In [27]:
#Dropping rows where our query did not return a FIPS (address not found in database)
schools_df_clean = schools_df[schools_df.FIPS != 'null']
schools_df_clean.shape

(373, 6)

In [32]:
schools_df_clean.index = range(len(schools_df_clean))

In [33]:
schools_df_clean.tail()

,School,Street,Zip,City,Rating,FIPS
368,Cook Jr Elementary School,7115 Lockwood Drive,77016,Houston,2,48201230400
369,Admin Services,4250 Cook Road,77072,Houston,1,48201452500
370,Sneed Elementary School,9855 Pagewood Lane,77042,Houston,1,48201432400
371,Highland Hts Elementary School,865 Paul Quinn Street,77091,Houston,1,48201530800
372,Pro-Vision Middle,4590 Wilmington Street,77051,Houston,1,48201331400


In [35]:
schools_df_clean.to_csv(r'../data/cleandata/school_ratings_data.csv', index=False)